Importing the required libraries

In [1]:
import pandas as pd

Loading the saved statistics CSV as a dataframe

In [3]:
data = pd.read_csv('data/consolidated_footnotes.csv')
data.head()

,page_file,footnote_text,cluster_label,word_count
0,0109_the_land_of_the_perumauls_page_109.jpg,* Repelim is known as Eddapully or Ellengaloor...,3,13
1,0359_the_land_of_the_perumauls_page_359.jpg,"* It is curious that it has been asserted, and...",0,36
2,0065_the_land_of_the_perumauls_page_65.jpg,"* This appears to have been an ancient symbol,...",3,15
3,0258_the_land_of_the_perumauls_page_258.jpg,* History is silent respecting what became of ...,0,46
4,0258_the_land_of_the_perumauls_page_258.jpg,"† It seems that after the advent of the Dutch,...",6,68
